<a href="https://colab.research.google.com/github/Shahid-TheMatrixMaker/Sentiment-Analysis-using-NN/blob/main/Twitter_Sentiment_analysis_using_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import library
import statsmodels.api as sm

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/kazanova/sentiment140")

Skipping, found downloaded files in "./sentiment140" (use force=True to force download)


In [ ]:
df = pd.read_csv("/content/sentiment140/training.1600000.processed.noemoticon.csv",
                 encoding='latin-1',
                 header=None)

In [ ]:
df.head(4)

,text,target
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0
1,is upset that he can't update his Facebook by ...,0
2,@Kenichan I dived many times for the ball. Man...,0
3,my whole body feels itchy and like its on fire,0


In [ ]:
# Optional: Rename columns for easier use
df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']

ValueError: Length mismatch: Expected axis has 2 elements, new values have 6 elements

In [ ]:
df.head(4)

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire


**Libraries are important and we call them to perform the different actions on our data and for training the models.
Its a first step to load the library to perform the specific task**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.cm as cm
from matplotlib import rcParams
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import re
import string
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

We are uisng the following versions of the libraries:
numpy == 1.18.5

pandas == 1.1.3

tensorflow ==1.7.0

keras == 2.4.3

nltk ==3.5

seaborn ==0.11.0

In [ ]:
# last five records of data
df.tail(5)

,target,ids,date,flag,user,text
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [ ]:
# Column/Fetures in data
df.columns

Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')

In [ ]:
# length of the data
print('lenght of data is', len(df))

lenght of data is 1600000


In [ ]:
# Shape of data
df.shape

(1600000, 6)

In [ ]:
# Data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   ids     1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


In [ ]:
# data types of all columns
df.dtypes

,0
target,int64
ids,int64
date,object
flag,object
user,object
text,object


In [ ]:
# checking null values
np.sum(df.isnull().any(axis=1))

np.int64(0)

It counts how many rows have at least one NaN (null) value in the DataFrame df

In [ ]:
df.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


Rows and columns in the dataset

In [ ]:
print('Count of columns in the data is:  ', len(df.columns))

Count of columns in the data is:   6


In [ ]:
print('Count of rows in the data is:  ', len(df))

Count of rows in the data is:   1600000


Data Preparation 📝
Selection of interested coloumns

Assinged 1 to class 4

Took one fourth data so we can run on our machine easily

Combined positive and negative tweets

We will convert the text in lower case for the further working on tweet text.

We will clean and remove the stop words(of, a, in etc) from statement because these words are not useuseful to support the labels of sentiments data

We will clean and remove the punctuations because these are the noise in the data and not meaningfull

We will clean and remove repeating characters in the words

We will clean and remove emails

We will clean and remove URL's

We will clean and remove the numbers in the data

We will apply tokenization(to separate the sentence into words)

We will apply stemming and lemmatization on the text. The concept of both is following:

STEMMING VS LEMMATIZATION
Stemming
Produced by “stemmers”

Produces a word’s “stem”

Examples:

am → am

the goin → the go

having → hav

Lemmatization
Produced by “lemmatizers”

Produces a word’s “lemma”

Examples:

am → be

the going → the going

having → have

We will apply stemming on the tweet text.

We will apply lemmatization on the tweet text.

Separated input feature and labels

Extracted features from input feature

Separated the 70% data for training and 30% data for testing

In [ ]:
# Selecting the text and label/target coloumn

df=df[['text','target']]

Assigning 1 to Positive sentment 4
In the Sentiment140 dataset, the target column represents sentiment labels:

0 = Negative sentiment

4 = Positive sentiment

These are the original labels in the dataset. But for many binary classification tasks, we prefer to have:

0 = Negative

1 = Positive
So this line
df.loc[df['target'] == 4, 'target'] = 1
is used to convert the '4' (positive) values into '1', making the sentiment labels binary: 0 (negative) and 1 (positive).

In [ ]:
df['target'][df['target']==4]=1

In [ ]:
# Separating positive and negative tweets
data_pos = df[df['target'] == 1]
data_neg = df[df['target'] == 0]

taking one fourth data so we can run on our machine easily

You are reducing the dataset size to 25% (¼th) of the total rows to make processing faster and less memory-intensive — especially helpful when:

Your machine has limited RAM or CPU power.

You're doing quick experimentation or prototyping.

You're avoiding long training times in the early stages of model development.

If your dataset has 1,000,000 rows, taking one-fourth means keeping only:

1,000,000 × 1/4 = 250,000 rows
💻 In code:
df = df.sample(frac=0.25, random_state=42)
This will:

Randomly sample 25% of the rows from the original df.

random_state=42 ensures reproducibility (same sample every time you run the code).



In [ ]:
data_pos = data_pos.iloc[:int(20000)]
data_neg = data_neg.iloc[:int(20000)]

Combining positive and negative tweets

In [ ]:
data = pd.concat([data_pos, data_neg])

Making text lowercase is a standard preprocessing step in NLP pipelines to improve model performance, reduce noise, and make analysis easier.

In [ ]:
# Making statement text in lower case
data['text']=data['text'].str.lower()

In [ ]:
data['text'].tail()

,text
19995,"not much time off this weekend, work trip to m..."
19996,one more day of holidays
19997,feeling so down right now .. i hate you damn h...
19998,"geez,i hv to read the whole book of personalit..."
19999,i threw my sign at donnie and he bent over to ...


Stop words are common words in a language that carry little meaning in text analysis and are usually removed to focus on more important words.

In [ ]:
# Cleaning and removing Stop words of english
import nltk
nltk.download('stopwords')

stopwords_list = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"a, about, above, after, again, against, ain, all, am, an, and, any, are, aren, aren't, as, at, be, because, been, before, being, below, between, both, but, by, can, couldn, couldn't, d, did, didn, didn't, do, does, doesn, doesn't, doing, don, don't, down, during, each, few, for, from, further, had, hadn, hadn't, has, hasn, hasn't, have, haven, haven't, having, he, he'd, he'll, her, here, hers, herself, he's, him, himself, his, how, i, i'd, if, i'll, i'm, in, into, is, isn, isn't, it, it'd, it'll, it's, its, itself, i've, just, ll, m, ma, me, mightn, mightn't, more, most, mustn, mustn't, my, myself, needn, needn't, no, nor, not, now, o, of, off, on, once, only, or, other, our, ours, ourselves, out, over, own, re, s, same, shan, shan't, she, she'd, she'll, she's, should, shouldn, shouldn't, should've, so, some, such, t, than, that, that'll, the, their, theirs, them, themselves, then, there, these, they, they'd, they'll, they're, they've, this, those, through, to, too, under, until, up, 

In [ ]:
# Cleaning and removing the above stop words list from the tweet text
STOPWORDS = set(stopwords.words('english'))
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
data['text'] = data['text'].apply(lambda text: cleaning_stopwords(text))
data['text'].head()

,text
800000,love @health4uandpets u guys r best!!
800001,im meeting one besties tonight! cant wait!! - ...
800002,"@darealsunisakim thanks twitter add, sunisa! g..."
800003,sick really cheap hurts much eat real food plu...
800004,@lovesbrooklyn2 effect everyone


In [ ]:
# Cleaning and removing punctuations
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [ ]:
data['text']= data['text'].apply(lambda x: cleaning_punctuations(x))
data['text'].tail()

,text
19995,much time weekend work trip malmï¿½ frisat tod...
19996,one day holidays
19997,feeling right hate damn humprey
19998,geezi hv read whole book personality types emb...
19999,threw sign donnie bent get thingee made sad face


In [ ]:
# Cleaning and removing repeating characters
def cleaning_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
data['text'] = data['text'].apply(lambda x: cleaning_repeating_char(x))
data['text'].tail()

,text
19995,much time wekend work trip malmï¿½ frisat toda...
19996,one day holidays
19997,feling right hate damn humprey
19998,gezi hv read whole bok personality types embar...
19999,threw sign donie bent get thinge made sad face


In [ ]:
# Cleaning and removing email
def cleaning_email(data):
    return re.sub('@[^\s]+', ' ', data)

In [ ]:
data['text']= data['text'].apply(lambda x: cleaning_email(x))
data['text'].tail()

,text
19995,much time wekend work trip malmï¿½ frisat toda...
19996,one day holidays
19997,feling right hate damn humprey
19998,gezi hv read whole bok personality types embar...
19999,threw sign donie bent get thinge made sad face


In [ ]:
# Cleaning and removing URL's
def cleaning_URLs(data):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',data)

In [ ]:
data['text'] = data['text'].apply(lambda x: cleaning_URLs(x))
data['text'].tail()

,text
19995,much time wekend work trip malmï¿½ frisat toda...
19996,one day holidays
19997,feling right hate damn humprey
19998,gezi hv read whole bok personality types embar...
19999,threw sign donie bent get thinge made sad face


In [ ]:
# Cleaming and removing numeric numbers
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)

In [ ]:
data['text'] = data['text'].apply(lambda x: cleaning_numbers(x))
data['text'].tail()

TypeError: expected string or bytes-like object, got 'list'

Getting tokenization of tweet text